# Случайные леса
__Суммарное количество баллов: 10__

__Решение отправлять на `ml.course.practice@gmail.com`__

__Тема письма: `[ML][MS][HW09] <ФИ>`, где вместо `<ФИ>` указаны фамилия и имя__

В этом задании вам предстоит реализовать ансамбль деревьев решений, известный как случайный лес, применить его к публичным данным пользователей социальной сети Вконтакте, и сравнить его эффективность с ансамблем, предоставляемым библиотекой CatBoost.

В результате мы сможем определить, какие подписки пользователей больше всего влияют на определение возраста и пола человека. 

In [1]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas
import random
import matplotlib.pyplot as plt
import matplotlib
import copy
from catboost import CatBoostClassifier
from math import sqrt
from typing import Callable, Union, NoReturn, Optional, Dict, Any, List
from scipy.stats import mode 

In [2]:
def gini(x):
    _, counts = np.unique(x, return_counts=True)
    proba = counts / len(x)
    return np.sum(proba * (1 - proba))
    
def entropy(x):
    _, counts = np.unique(x, return_counts=True)
    proba = counts / len(x)
    return -np.sum(proba * np.log2(proba))

def gain(left_y, right_y, criterion):
    y = np.concatenate((left_y, right_y))
    return criterion(y) - (criterion(left_y) * len(left_y) + criterion(right_y) * len(right_y)) / len(y)

### Задание 1 (2 балла)
Random Forest состоит из деревьев решений. Каждое такое дерево строится на одной из выборок, полученных при помощи bagging. Элементы, которые не вошли в новую обучающую выборку, образуют out-of-bag выборку. Кроме того, в каждом узле дерева мы случайным образом выбираем набор из `max_features` и ищем признак для предиката разбиения только в этом наборе.

Сегодня мы будем работать только с бинарными признаками, поэтому нет необходимости выбирать значение признака для разбиения.

#### Методы
`predict(X)` - возвращает предсказанные метки для элементов выборки `X`

#### Параметры конструктора
`X, y` - обучающая выборка и соответствующие ей метки классов. Из нее нужно получить выборку для построения дерева при помощи bagging. Out-of-bag выборку нужно запомнить, она понадобится потом.

`criterion="gini"` - задает критерий, который будет использоваться при построении дерева. Возможные значения: `"gini"`, `"entropy"`.

`max_depth=None` - ограничение глубины дерева. Если `None` - глубина не ограничена

`min_samples_leaf=1` - минимальное количество элементов в каждом листе дерева.

`max_features="auto"` - количество признаков, которые могут использоваться в узле. Если `"auto"` - равно `sqrt(X.shape[1])`

In [24]:
class DecisionTreeLeaf:
    
    def __init__(self, y):
        self.y = mode(y)[0][0]

class DecisionTreeNode:
    
    def __init__(self, split_dim: int, 
                 left: Union['DecisionTreeNode', DecisionTreeLeaf], 
                 right: Union['DecisionTreeNode', DecisionTreeLeaf]):
        self.split_dim = split_dim
        self.left = left
        self.right = right
        
    def query(self, datapoint):
        
        if datapoint[self.split_dim] == 1:
            if isinstance(self.left, DecisionTreeLeaf):
                return self.left.y
            return self.left.query(datapoint)
        
        if isinstance(self.right, DecisionTreeLeaf):
            return self.right.y
        return self.right.query(datapoint)

In [30]:
class DecisionTree:

    def __init__(self, 
                 X, y,
                 criterion : str = "gini", 
                 max_depth : Optional[int] = None, 
                 min_samples_leaf: int = 1,
                 max_features: Optional[int] = "auto"):
        
        
        self.criterion = entropy if criterion == 'entropy' else gini
        self.min_samples_leaf = min_samples_leaf
        self.max_depth = max_depth if max_depth is not None else np.inf 
        self.max_features = round(sqrt(X.shape[1])) if "auto" else max_features
        n = X.shape[0]
        self.train_indices = np.random.choice(np.arange(n), size=n, replace=True)
        self.oob_indices = np.setdiff1d(np.arange(n), self.train_indices)
        self.root = self._build_tree(X[self.train_indices], y[self.train_indices])
        
    def _build_tree(self, X, y, depth=0):
        n, m = X.shape
        
        if n <= 2 * self.min_samples_leaf or depth == self.max_depth:
            return DecisionTreeLeaf(y)
        
        feature_indices = np.random.choice(np.arange(m), size=self.max_features, replace=False)
        
        ig = -1
        best_feature = 0
        feature_found = False
        
        for feature in feature_indices:
            cond = X[:, feature] == 1
            y_left, y_right = y[cond], y[~cond]
                
            if y_left.shape[0] >= self.min_samples_leaf and y_right.shape[0] >= self.min_samples_leaf:
                current_ig = gain(left_y=y_left, right_y=y_right, criterion=self.criterion)
                    
                if current_ig > ig:
                    best_feature = feature
                    ig = current_ig
                    feature_found = True
                        
        if not feature_found:
            return DecisionTreeLeaf(y)
        
        cond = X[:, best_feature] == 1
        X_left, y_left, X_right, y_right = X[cond], y[cond], X[~cond], y[~cond]
        
        left_child = self._build_tree(X_left, y_left, depth + 1)
        right_child = self._build_tree(X_right, y_right, depth + 1)
        root = DecisionTreeNode(split_dim=best_feature, left=left_child, right=right_child)
    
        return root
    
    def predict(self, X : np.ndarray) -> list:
        return [self.root.query(x) for x in X]

### Задание 2 (2 балла)
Теперь реализуем сам Random Forest. Идея очень простая: строим `n` деревьев, а затем берем модальное предсказание.

#### Параметры конструктора
`n_estimators` - количество используемых для предсказания деревьев.

Остальное - параметры деревьев.

#### Методы
`fit(X, y)` - строит `n_estimators` деревьев по выборке `X`.

`predict(X)` - для каждого элемента выборки `X` возвращает самый частый класс, который предсказывают для него деревья.

In [31]:
class RandomForestClassifier:
    def __init__(self, criterion="gini", max_depth=None, min_samples_leaf=1, max_features="auto", n_estimators=10):
        self.params = {'criterion': criterion, 'max_depth': max_depth, 'min_samples_leaf': min_samples_leaf, 'max_features': max_features}
        self.n = n_estimators
        self.estimators = []
    
    def fit(self, X, y):
        self.X = X
        self.y = y
        for _ in range(self.n):
            self.estimators.append(DecisionTree(X, y, **self.params))
    
    def predict(self, X):
        preds = [estimator.predict(X) for estimator in self.estimators]
        return mode(preds, axis=0)[0][0]

### Задание 3 (2 балла)
Часто хочется понимать, насколько большую роль играет тот или иной признак для предсказания класса объекта. Есть различные способы посчитать его важность. Один из простых способов сделать это для Random Forest - посчитать out-of-bag ошибку предсказания `err_oob`, а затем перемешать значения признака `j` и посчитать ее (`err_oob_j`) еще раз. Оценкой важности признака `j` для одного дерева будет разность `err_oob_j - err_oob`, важность для всего леса считается как среднее значение важности по деревьям.

Реализуйте функцию `feature_importance`, которая принимает на вход Random Forest и возвращает массив, в котором содержится важность для каждого признака.

In [32]:
def feature_importance(rfc):
    X, y = rfc.X, rfc.y
    n, m = X.shape
    importances = np.zeros((rfc.n, m))
    
    for i in range(rfc.n):
        
        idx = rfc.estimators[i].oob_indices
        X_oob, y_oob = X[idx], y[idx]
        err_oob = 1 - np.mean(rfc.estimators[i].predict(X_oob) == y_oob)
        
        for col in range(m):
            X_oob_shuf = X_oob.copy()
            X_oob_shuf[:, col] = np.random.permutation(X_oob_shuf[:, col])
            err_oob_j = 1 - np.mean(rfc.estimators[i].predict(X_oob_shuf) == y_oob)
            importances[i, col] = err_oob_j - err_oob

    return np.mean(importances, axis=0)

def most_important_features(importance, names, k=20):
    # Выводит названия k самых важных признаков
    idicies = np.argsort(importance)[::-1][:k]
    return np.array(names)[idicies]

Наконец, пришло время протестировать наше дерево на простом синтетическом наборе данных. В результате точность должна быть примерно равна `1.0`, наибольшее значение важности должно быть у признака с индексом `4`, признаки с индексами `2` и `3`  должны быть одинаково важны, а остальные признаки - не важны совсем.

In [48]:
%%time

def synthetic_dataset(size):
    X = [(np.random.randint(0, 2), np.random.randint(0, 2), i % 6 == 3, 
          i % 6 == 0, i % 3 == 2, np.random.randint(0, 2)) for i in range(size)]
    y = [i % 3 for i in range(size)]
    return np.array(X), np.array(y)

X, y = synthetic_dataset(1000)
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X, y)
print("Accuracy:", np.mean(rfc.predict(X) == y))
print("Importance:", feature_importance(rfc))

Accuracy: 1.0
Importance: [7.76858824e-04 4.47371646e-05 1.80787493e-01 1.65989432e-01
 3.39516761e-01 1.76546689e-03]
Wall time: 1.37 s


### Задание 4 (1 балл)
Теперь поработаем с реальными данными.

Выборка состоит из публичных анонимизированных данных пользователей социальной сети Вконтакте. Первые два столбца отражают возрастную группу (`zoomer`, `doomer` и `boomer`) и пол (`female`, `male`). Все остальные столбцы являются бинарными признаками, каждый из них определяет, подписан ли пользователь на определенную группу/публичную страницу или нет.\
\
Необходимо обучить два классификатора, один из которых определяет возрастную группу, а второй - пол.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются. Лес должен строиться за какое-то разумное время.

In [35]:
def read_dataset(path):
    dataframe = pandas.read_csv(path, header=0)
    dataset = dataframe.values.tolist()
    random.shuffle(dataset)
    y_age = [row[0] for row in dataset]
    y_sex = [row[1] for row in dataset]
    X = [row[2:] for row in dataset]
    
    return np.array(X), np.array(y_age), np.array(y_sex), list(dataframe.columns)[2:]

In [36]:
X, y_age, y_sex, features = read_dataset("vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)

#### Возраст

In [37]:
%%time

rfc = RandomForestClassifier(n_estimators=10)

rfc.fit(X_train, y_age_train)
print("Accuracy:", np.mean(rfc.predict(X_test) == y_age_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc), features, 20)):
    print(str(i+1) + ".", name)

Accuracy: 0.7225725094577553
Most important features:
1. ovsyanochan
2. 4ch
3. styd.pozor
4. rhymes
5. mudakoff
6. rapnewrap
7. dayvinchik
8. iwantyou
9. tumblr_vacuum
10. pixel_stickers
11. xfilm
12. reflexia_our_feelings
13. pozor
14. ultrapir
15. pravdashowtop
16. ohhluul
17. i_d_t
18. nenormalnoo
19. fuck_humor
20. bot_maxim
Wall time: 1min 31s


#### Пол

In [39]:
%%time

rfc = RandomForestClassifier(n_estimators=10)
rfc.fit(X_train, y_sex_train)
print("Accuracy:", np.mean(rfc.predict(X_test) == y_sex_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc), features, 20)):
    print(str(i+1) + ".", name)

Accuracy: 0.8549810844892812
Most important features:
1. 40kg
2. mudakoff
3. girlmeme
4. 9o_6o_9o
5. zerofat
6. modnailru
7. cook_good
8. be.beauty
9. reflexia_our_feelings
10. 4ch
11. femalemem
12. psy.people
13. beauty
14. thesmolny
15. i_d_t
16. woman.blog
17. rapnewrap
18. be.women
19. sh.cook
20. combovine
Wall time: 1min 31s


### CatBoost
В качестве аьтернативы попробуем CatBoost. 

Устаниовить его можно просто с помощью `pip install catboost`. Туториалы можно найти, например, [здесь](https://catboost.ai/docs/concepts/python-usages-examples.html#multiclassification) и [здесь](https://github.com/catboost/tutorials/blob/master/python_tutorial.ipynb). Главное - не забудьте использовать `loss_function='MultiClass'`.\
\
Сначала протестируйте CatBoost на синтетических данных. Выведите точность и важность признаков.

In [41]:
X, y = synthetic_dataset(1000)
cb_clf = CatBoostClassifier(loss_function="MultiClass", verbose=False, depth=6)
cb_clf.fit(X, y)
y_pred = cb_clf.predict(X).flatten()

print("Accuracy:", np.mean(y_pred == y))
print("Importance:", cb_clf.feature_importances_)

Accuracy: 1.0
Importance: [2.91197381e-03 5.83727700e-03 2.78666677e+01 2.78606166e+01
 4.42601799e+01 3.78655607e-03]


### Задание 5 (3 балла)
Попробуем применить один из используемых на практике алгоритмов. В этом нам поможет CatBoost. Также, как и реализованный ними RandomForest, применим его для определения пола и возраста пользователей сети Вконтакте, выведите названия наиболее важных признаков так же, как в задании 3.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются.

In [51]:
X, y_age, y_sex, features = read_dataset("vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)
X_train, X_eval, y_age_train, y_age_eval, y_sex_train, y_sex_eval = train_test_split(X_train, y_age_train, y_sex_train, train_size=0.8)

#### Возраст

In [55]:
cb_clf = CatBoostClassifier(
    loss_function="MultiClass",
    verbose=False,
    learning_rate=1e-2,
    max_depth=8,
    min_data_in_leaf=20,
)
cb_clf.fit(X_train, y_age_train)
y_pred = cb_clf.predict(X_test).flatten()

print("Accuracy:", np.mean(y_pred == y_age_test))
print("Most important features:")
for i, name in enumerate(most_important_features(cb_clf.feature_importances_, features, 10)):
    print(str(i+1) + ".", name)

Accuracy: 0.7440100882723834
Most important features:
1. ovsyanochan
2. styd.pozor
3. 4ch
4. rhymes
5. mudakoff
6. dayvinchik
7. leprum
8. xfilm
9. thesmolny
10. fuck_humor


#### Пол

In [56]:
cb_clf = CatBoostClassifier(
    loss_function="MultiClass",
    verbose=False,
    learning_rate=1e-2,
    max_depth=8,
    min_data_in_leaf=20,
)
cb_clf.fit(X_train, y_sex_train)
y_pred = cb_clf.predict(X_test).flatten()


print("Accuracy:", np.mean(y_pred == y_sex_test))
print("Most important features:")
for i, name in enumerate(most_important_features(cb_clf.feature_importances_, features, 10)):
    print(str(i+1) + ".", name)

Accuracy: 0.8713745271122321
Most important features:
1. 40kg
2. girlmeme
3. modnailru
4. mudakoff
5. femalemem
6. academyofman
7. thesmolny
8. i_d_t
9. 9o_6o_9o
10. igm
